In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.animation as animation
from IPython.display import HTML

plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
from scipy.ndimage import convolve

# Mesh parameters
nx, ny = 101, 101
vx, dx = np.linspace(0, 1, nx, endpoint=True, retstep=True)
vy, dy = np.linspace(0, 1, ny, endpoint=True, retstep=True)
x, y = np.meshgrid(0.5 * (vx[:-1] + vx[1:]), 0.5 * (vy[:-1] + vy[1:]))

def charge_density(x, y):
    sigma = 0.05 
    x0, y0 = 0.5, 0.5 
    rho0 = 50.0  
    return rho0 * np.exp(-((x - x0)**2 + (y - y0)**2) / (2 * sigma**2))
    

def solve_poisson(rho, dx, dy, tol=1e-6, max_iter=10000):
    phi = np.zeros_like(rho)
    laplacian_kernel = np.array([[0, 1, 0],
                                 [1, -4, 1],
                                 [0, 1, 0]])

    for i in range(max_iter):
        phi_new = convolve(phi, laplacian_kernel, mode='constant', cval=0) * (dx * dy)
        phi_new += 4 * np.pi * rho * dx * dy
        if np.linalg.norm(phi_new - phi, ord=np.inf) < tol:
            break
        phi = phi_new
        
    #E = -grad(phi)
    ex, ey = np.gradient(-phi, dx, dy)
    return ex, ey


def update_magnetic(ex, ey, hz) : 
    return hz + dt * ((ex[:, 1:] - ex[:, :-1]) / dy - (ey[1:, :] - ey[:-1, :]) / dx)

def update_electric(hz, ex, ey):
    ex[:, 1:-1] += dt * (hz[:, 1:] - hz[:, :-1]) / dy
    ey[1:-1, :] += -dt * (hz[1:, :] - hz[:-1, :]) / dx

    # periodic boundary conditions
    ex[:, 0] += dt * (hz[:, 0] - hz[:, -1]) / dy
    ex[:, -1] = ex[:, 0]
    ey[0, :] += - dt * (hz[0, :]-hz[-1, :]) / dx
    ey[-1, :] = ey[0, :]
    return ex, ey



def update(i, ax):
    ax.cla()
    rho = charge_density(x, y) * np.sin(np.pi * dt * i)
    ex, ey = solve_poisson(rho, dx, dy)
    wframe =  ax.plot_wireframe(x, y, rho, rstride=2, cstride=2)
    ax.set_zlim(-1, 10)
    return wframe

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
hz = np.zeros((nx - 1, ny - 1))
dt = 0.01

ani = animation.FuncAnimation(fig, update, frames=range(200), fargs=(ax,), interval=20, blit=False)
HTML(ani.to_html5_video())

In [ ]:
#Initialize Ex, Ey when time = 0
ex = np.zeros((nx - 1, ny), dtype = np.double)  
ey = np.zeros((nx, ny - 1), dtype = np.double) 
dt = 0.001     # time step
m, n = 2, 2
omega = np.sqrt((m * np.pi)**2 + (n * np.pi)**2)

# Create the staggered grid for Bz
x, y = np.meshgrid(0.5 * (vx[:-1] + vx[1:]), 0.5 * (vy[:-1] + vy[1:]))

#Initialize Bz when time = - dt / 2
hz = - np.cos(m * np.pi * y) * np.cos(n * np.pi * x) 

fig = plt.figure()
axs = fig.add_subplot(111, projection='3d')
axs.plot_wireframe(x, y, hz, rstride=2, cstride=2)

In [ ]:



def update(i, ax):
    ax.cla()

    global ex, ey, hz

    #loops before plotting
    for j in range(5):
        hz = update_magnetic(ex, ey, hz)
        ex, ey = update_electric(hz, ex, ey)
    
    wframe =  ax.plot_wireframe(x, y, hz, rstride=2, cstride=2)
    ax.set_zlim(-1, 1)
    return wframe



# Create a figure and a 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ani = animation.FuncAnimation(fig, update, frames=range(200), fargs=(ax,), interval=20, blit=False)
HTML(ani.to_html5_video())